In [2]:
import numpy as np
import numpy.linalg as linalg
import math
from copy import deepcopy
maxiter = 10

In [3]:
# допоміжна функція для виводу матриці
def print_matrix(A, text):
    '''As argument takes matrix and text you want to ouput
    with matrix'''
    n,m = np.shape(A)
    print(text)
    for i in range(0,n):
        for j in range(0,m):
            print('{:>6}'.format(round(A[i,j], 3)), end = ' ')
        print()
# допоміжна функція для виводу вектора
def print_vector(b, text):
    '''As argument takes matrix and text you want to ouput
    with matrix'''
    n=len(b)
    print(text)
    for i in range(0,n):
        print('{:>6}'.format(round(b[i], 3),''))
        


In [4]:
def buildMatrixBandC(A,b):
    n = np.shape(A)[0]
    I = np.eye(n,n)
    B = np.zeros((n,n))
    c = np.zeros((n,1))
    for i in range(0,n):
        if A[i,i] >= 0:
            B[i] = (I[i] - A[i])
            c[i] = b[i]
        else :
            B[i] = I[i] + A[i]
            c[i] =-b[i]
    return B,c

# Метод простих ітерацій

Систему лінійних алгебраїчних рівнянь $Ax = b$ перетворимо у еквівалентну систему $x = Bx +c$, де х - той самий вектор невідомих, а В і с - деякі нові матриця і вектор відповідно. Складаємо ітераційну послідовність $$x^{(k+1)} = Bx^{(k)} , k = 0,1,2,...$$ яка починається з деякого вектора $x^{(0)} = (x_1^{(0)}, x_2^{(0)},...,x_n^{(0)})$. Ітераційний процес, який визначається цими формулами назівається **методом простих ітерацій**

### Збіжність методу простих ітерацій
Метод простих ітрацій збігається при довільному початковому векторі $x^{(0)}$ до розв'язку системи $x^*$ тоді і тільки тоді, коли всі власні значення $\lambda_b$ матриці В за модулем менше одиниці.

**Теорема** Нехай $\|B\| \leq q < 1$. Тоді при довільному початковому векторі $x^{(0)}$ МПІ збігається до єдиного розв'язку $x^*$ задачі про нерухому точку для всіх $k \in N$ і є справедливими оцінки похібки :
_апостеріорна_ $$\|x^* - x^{(k)}\| \leq \frac{q}{1-q}\|x^{(k)} - x^{(k-1)}\|$$
_апріорна_ $$\|x^* - x^{(k)}\| \leq \frac{q^k}{1-q}\|x^{(1)} - x^{(0)}\|$$

In [46]:
def SimpleIteratons(A,b,eps):

    maxB = 1
    n = np.shape(A)[0]
    B,c = buildMatrixBandC(A,b)
    q = linalg.norm(B)
    x_curr = np.zeros(n)
    x_next = c
    cnt = 0
    delta = 1
    apriori = estimateApriori(B,c,eps)
    if apriori > 0:
        print("Попередня оцінка достатньої кількості операцій: {}".format(apriori))
    else :
        print("Вимога до малості норми матриці не виконана, процес може розходитись")
    while ( delta > (1-q)/q*eps) and (cnt< maxiter):
        cnt += 1
        x_curr = x_next
        x_next = np.dot(B,x_curr) + c
        print_matrix(x_next,"Значення на {} ітерації".format(cnt))
        delta = linalg.norm(x_next - x_curr)
        print("Похибка наближення {}".format(delta))

    if (cnt == maxiter):
        err = linalg.norm(x_next - x_curr)
        if (err > linalg.norm(c)):
            print("Ітераційний процес розходиться")
            return
        else:
            print("Виконано {} ітерацій,апостеріорна оцінка похибки :{}".format(cnt,err))
        
    
    return x_next



# Метод Якобі
Представимо матрицю А у вигляді $A = L + D + R $, де  D - діагональна. L і R - відповідно діва та права строго трикутні матриці. Тоді система $Ax = b$ запишеться у вигляді $Lx +Dx +Rx = b$.
$$x^{(k+1)} = - D^{-1}(L+R)x^{(k)} + D^{-1}b, k = 0,1,2...$$
Оберненою до діагональної матриці є матриця з елементами $\frac{1}{a_{ii}}$ на діагоналі.Тому це представлення рівнозначне вираженнб діагональних невідомих через інші:
$$\begin{cases} x_1^{(k+1)} = - (a_{12}x_2^{(k)} + a_{13}x_3^{(k)}  + ... + a_{1n}x_n^{(k)}  - b_1)/a_{11} \\ x_2^{(k+1)} = - (a_{21}x_1^{(k)}  + a_{23}x_3^{(k)}  + ... + a_{2n}x_n^{(k)}  - b_2)/a_{22}\\ ..........................................\\x_n^{(k+1)} = - (a_{n1}x_1^{(k)}  + a_{n2}x_2 ^{(k)} + ... + a_{n,n-1}x_{n-1}^{(k)}  - b_n)/a_{nn}  \end{cases}$$
Метод Якобі збігається,якщо виконується умова діагонального домінування.

In [70]:
def SimpleIteratonsJacobi(A,b,eps):
    n = np.shape(A)[0]
    q = 0.5
    cnt  = 0
    x_curr = np.zeros(n)
    x_next = np.zeros(n)
    tostop = False
    apriori = estimateApriori(A,b,eps)

    while not tostop:
        cnt += 1

        x_curr = deepcopy(x_next)
        for i in range(0,n):
            sum = 0;
            for j in range (0,n):
                
                if i!= j:
                    sum += A[i,j]*x_curr[j]
            x_next[i] = 1/A[i,i]*(b[i] - sum)
        print_vector(x_next,"Значення на {} ітерації".format(cnt))
        delta = linalg.norm(x_next - x_curr)
        print("Похибка наближення {}".format(delta))

        
        if (linalg.norm(x_next - x_curr) < eps) or (cnt == maxiter):

            tostop = True
    if (cnt == maxiter):
        err = linalg.norm(x_next - x_curr)
        if (err > linalg.norm(b)):
            print("Ітераційний процес розходиться")
            return
        else:
                print("Виконано {} ітерацій,апостеріорна оцінка похибки :{}".format(cnt,err))    
    return x_next

# Метод Зейделя
Це модифікація методу простих ітерацій, в якій при обчисленні i-ї координати (k+1)-го наближення до шуканого вектора $x^*$ використовують уже знайдені на цій ітерації i-1 компонент.

$$\begin{cases}
x_1^{(k+1)} = - (a_{12}x_2^{(k)} + a_{13}x_3^{(k)}  + ... + a_{1n}x_n^{(k)}  - b_1)/a_{11} \\
x_2^{(k+1)} = - (a_{21}x_1^{(k+1)}  + a_{23}x_3^{(k)}  + ... + a_{2n}x_n^{(k)}  - b_2)/a_{22}\\ ..........................................\\
x_n^{(k+1)} = - (a_{n1}x_1^{(k+1)}  + a_{n2}x_2 ^{(k+1)} + ... + a_{n,n-1}x_{n-1}^{(k+1)}  - b_n)/a_{nn}  \end{cases}$$

In [71]:
def SimpleIteratonsZeidel(A,b,eps):
    n = np.shape(A)[0]
    q = 0.5
    cnt  = 0
    x_curr = np.zeros(n)
    x_next = np.zeros(n)
    tostop = False

    while not tostop:
        cnt += 1
        x_curr = deepcopy(x_next)
        for i in range(0,n):
            sum = 0;
            for j in range (0,n):
                if i > j:
                    sum += A[i,j]*x_next[j]
                elif i < j:
                    sum += A[i,j]*x_curr[j]
            x_next[i] = 1/A[i,i]*(b[i] - sum)
        print_vector(x_next,"Значення на {} ітерації".format(cnt))
        delta = linalg.norm(x_next - x_curr)
        print("Похибка наближення {}".format(delta))
        if (linalg.norm(x_next - x_curr) < eps) or (cnt == maxiter):
            tostop = True
    if (cnt == maxiter):
        err = linalg.norm(x_next - x_curr)
        if (err > linalg.norm(b)):
            print("Ітераційний процес розходиться")
            return
        else:
                print("Виконано {} ітерацій,апостеріорна оцінка похибки :{}".format(cnt,err))        
    return x_next

In [72]:
def estimateApriori(B,c, eps):
    math.log(0.001/2.3,0.5)
    c_norm = linalg.norm(c)
    B_norm = linalg.norm(B)
    return math.ceil(math.log(eps/c_norm,B_norm)) + 1

Перевіримо роботу програми

In [89]:
A = np.matrix([ [1.1,  -0.2,0.1],
               [0.1,-1.2,-0.2],
               [0.2,-0.1,1.1]])
b = np.matrix([1.6,2.3,1.5]).T

A1 = np.matrix([ [7.8,  5.3,4.8],
               [3.3,1.1,1.8],
               [4.5,3.3,2.8]])
b1 = np.matrix([1.8,2.3,3.4]).T

In [91]:
x = SimpleIteratons(A1,b1,0.001)

Вимога до малості норми матриці не виконана, процес може розходитись
Значення на 1 ітерації
-38.95 
 -9.99 
-18.41 
Похибка наближення 47.82554442972918
Значення на 2 ітерації
407.975 
164.972 
244.78 
Похибка наближення 547.3779618956174
Значення на 3 ітерації
-4821.726 
-1801.119 
-2817.499 
Похибка наближення 6371.2506066363185
Значення на 4 ітерації
55859.459 
21165.605 
32716.355 
Похибка наближення 73975.2076144569
Значення на 5 ітерації
-649058.731 
-245339.914 
-380100.1 
Похибка наближення 859274.2702692258
Значення на 6 ітерації
7538383.195 
2850610.283 
4414569.586 
Похибка наближення 9980379.266803829
Значення на 7 ітерації
-87559172.438 
-33107948.526 
-51275960.166 
Похибка наближення 115922379.82005504
Значення на 8 ітерації
1016999110.368 
384552794.498 
595569237.808 
Похибка наближення 1346439083.851471
Значення на 9 ітерації
-11812456101.024 
-4466576969.42 
-6917544843.155 
Похибка наближення 15638902269.694328
Значення на 10 ітерації
137201774673.83 
51879343550.3 

In [11]:
r = np.dot(A,x) - b
print_matrix(r,"Нев'язка першої системи:")

Нев'язка першої системи:
   0.0 
   0.0 
   0.0 


In [76]:
res = SimpleIteratonsZeidel(A,b,0.001)

Значення на 1 ітерації
 1.455
-1.795
 0.936
Похибка наближення 2.493062911389281
Значення на 2 ітерації
 1.043
-1.986
 0.993
Похибка наближення 0.45703077370451317
Значення на 3 ітерації
 1.003
-1.999
   1.0
Похибка наближення 0.04230357111843943
Значення на 4 ітерації
   1.0
  -2.0
   1.0
Похибка наближення 0.0031843689301927604
Значення на 5 ітерації
   1.0
  -2.0
   1.0
Похибка наближення 0.0002833325256964141


In [77]:
SimpleIteratonsJacobi(A1,b1,0.001)

Значення на 1 ітерації
 0.231
 2.091
 1.214
Похибка наближення 2.4289184960048513
Значення на 2 ітерації
-1.937
-0.588
-1.621
Похибка наближення 4.462861993119961
Значення на 3 ітерації
 1.628
10.555
 5.021
Похибка наближення 13.453717378257267
Значення на 4 ітерації
-10.031
-11.01
-13.842
Похибка наближення 30.93201966152142
Значення на 5 ітерації
 16.23
54.835
30.311
Похибка наближення 83.51450811402346
Значення на 6 ітерації
-55.682
-96.199
-89.496
Похибка наближення 205.75839027211433
Значення на 7 ітерації
120.672
315.585
204.081
Похибка наближення 535.5882010389583
Значення на 8 ітерації
-339.794
-693.875
-564.662
Похибка наближення 1349.8159345279003
Значення на 9 ітерації
819.194
1945.464
1365.092
Похибка наближення 3468.9064034810403
Значення на 10 ітерації
-2161.744
-4689.278
-3608.216
Похибка наближення 8811.332522601951
Ітераційний процес розходиться


In [30]:
A2 = np.matrix([ [8.,  2.,-3.],
               [2.,10.,5.],
               [1.,-1.,16.]])
b2 = np.matrix([10.,12.,0.]).T

A3 = np.matrix([ [4.,  -1.,2.],
               [-1.,4.,2.],
               [3.,3.,4.]])
b3 = np.matrix([3.,3.,6.]).T

In [86]:
r =SimpleIteratonsJacobi(A2,b2,0.001)

Значення на 1 ітерації
  1.25
   1.2
   0.0
Похибка наближення 1.7327723451163457
Значення на 2 ітерації
  0.95
  0.95
-0.003
Похибка наближення 0.39052498719672224
Значення на 3 ітерації
 1.011
 1.012
   0.0
Похибка наближення 0.08695312499999987
Значення на 4 ітерації
 0.997
 0.998
   0.0
Похибка наближення 0.019834064312561957
Значення на 5 ітерації
 1.001
 1.001
   0.0
Похибка наближення 0.004476045417214398
Значення на 6 ітерації
   1.0
   1.0
  -0.0
Похибка наближення 0.000994023024315682


In [88]:
l = SimpleIteratonsZeidel(A3,b3,0.001)

Значення на 1 ітерації
  0.75
 0.938
 0.234
Похибка наближення 1.2232489078781146
Значення на 2 ітерації
 0.867
  0.85
 0.212
Похибка наближення 0.1481231573448976
Значення на 3 ітерації
 0.856
 0.858
 0.214
Похибка наближення 0.01388654600108415
Значення на 4 ітерації
 0.857
 0.857
 0.214
Похибка наближення 0.001301863687601639
Значення на 5 ітерації
 0.857
 0.857
 0.214
Похибка наближення 0.00012204972071265366
